# Homework 4: machine learning

---
### NOTE: Sometimes we refer to [the original work](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb) 

---

In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

You need to install `scikit v0.18`: `conda update scikit-learn`

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, KFold

Read and minimal cleanup: for the first part, we need the labels (i.e. colour ratings), so we can't use the points where they don't exist. 

Since we will later `aggregate` the players, it is **important** to note that this doesn't produce inconsistencies because `dyads` is constructed by a join between a `players` table and a `referees` table, so it is natural that the missing values are missing for all instances of a player.

In [3]:
dyads = pd.read_csv("CrowdstormingDataJuly1st.csv", index_col=0)
print(dyads.shape)

dyads.dropna(subset=['rater1'], inplace=True)
print(dyads.shape)

# since both values are missing at the same time, this should be 0:
print(dyads.rater2.isnull().sum())

# the groupby object user later on
group_players = dyads.groupby(level=0)

(146028, 27)
(124621, 27)
0


Let's see the numbers: 

<sub>yes, they were done by the other guys, but it's useful to have them at hand:</sub>

Also, they were excluding some referees that have been _carried over_, and that only removes ~3% of the data. Since we're not doing statistics on referees, we won't drop them. Every little data helps :)

In [4]:
print("Number of players: " , dyads.index.unique().size)
print("Number of referees: ", dyads.refNum.unique().size)

Number of players:  1585
Number of referees:  2978


The original analysis mentioned that "*the two raters disagree on 28742 or 19% of the time*". Since there are only 1585 players, it means they ran it on the `dyads` set. _**WHY?**_ That doesn't make sense, so let's just check that the ratings for each player are consistent:

for the group of each player, we check that the number of values in `raterX` is **exactly** one:

In [5]:
def build_player_consitency(player_df):
    """ Needs to return a Series of {col_name: col_value}. """
    return pd.Series({col+"_INconsistent" : (player_df[col].unique().size != 1) 
                                             for col in ['rater1', 'rater2'] })
consistency = group_players.apply(build_player_consitency)
print("Rater1 has been inconsistent %d times" % consistency.rater1_INconsistent.sum())
print("Rater2 has been inconsistent %d times" % consistency.rater2_INconsistent.sum())

Rater1 has been inconsistent 0 times
Rater2 has been inconsistent 0 times


OK, so they _ARE_ consistent. This means that their statistic doesn't account for players who have more matches than others, so the numbers are skewed. Let's check again, this time on _unique_ players

In [6]:
player_ratings = group_players.agg({'rater1':'first', 'rater2':'first'})
diffs = player_ratings.rater1 - player_ratings.rater2
print("The raters disagree for {p:.3f}% of the players".format(p=(diffs != 0).sum() / len(diffs) ))

print("Diffs std dev: ", diffs.std())

max_diff = diffs.abs().max()
num_occur = (diffs.abs() == max_diff).sum()
print("Max disagreement value {0}, occuring {1} times".format(max_diff * 4, num_occur)) # *4 to pass from float to int

The raters disagree for 0.239% of the players
Diffs std dev:  0.11594303556667578
Max disagreement value 2.0, occuring 2 times


So this means:
  1. that there is slightly more agreement between the raters for players who have more entries in `dyads` i.e. who played under more referees
  2. that if we use both labels, using `accuracy` as a measure of performance is not a very good idea. Keep in mind that the differences are not ordered, so it could have an impact double as big on the accuracy, i.e. at most $1 - 2 * \mathit{disagreementPercentage} = 1 - 2 * 0.24 \approx 0.5 $

#### Curiosity
Who are the 'controversial' guys :) ?

In [7]:
diffs[diffs.abs() == max_diff]

playerShort
kyle-walker    -0.5
mario-goetze   -0.5
dtype: float64

<img style='float:left' alt='Kyle-walker' src='http://www.thefootballsocial.co.uk/images/players/Tottenham%20Hotspur/Kyle%20Walker.jpg' /> <img alt='Mario_Goetze' src='http://i0.web.de/image/176/31756176,pd=2/mario-goetze.jpg' width=300/>

## Feature selection

In [8]:
dyads.columns

Index(['player', 'club', 'leagueCountry', 'birthday', 'height', 'weight',
       'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp'],
      dtype='object')

### Aggregation

Here's how we aggregate the data for each player:
<table>
<tr><th>Variable</th><th>Method</th><th>Fill NA</th></tr>
<tr><td>`height`, `weight`</td><td>first</td><td>average</td></tr>
<tr><td>`games`</td><td>sum</td></tr>
<tr><td>`goals`</td><td>sum ; percentage in games</td></tr>
<tr><td>`victories`, `ties`, `defeats`</td><td>sum ; percentage in games</td></tr>
<tr><td>`redCards`, `yellowCards`, `yellowReds`</td><td>sum ; percentage in games</td></tr>
</table>


In [9]:
players = group_players.agg({'height':'first', 'weight':'first', 'games':'sum', 
                             'victories':'sum','defeats':'sum', 'ties': 'sum', 'goals':'sum', 
                             'redCards':'sum', 'yellowReds': 'sum', 'yellowCards':'sum'})

# fill by average
av_height = players['height'].mean()
av_weight = players['weight'].mean()
players['height'].fillna(value=av_height, inplace=True)
players['weight'].fillna(value=av_weight, inplace=True)

print(players.shape)
players.head()

(1585, 10)


,height,defeats,victories,weight,goals,games,yellowReds,ties,yellowCards,redCards
playerShort,,,,,,,,,,
aaron-hughes,182.0,228,247,71.0,9,654,0,179,19,0
aaron-hunt,183.0,122,141,73.0,62,336,0,73,42,1
aaron-lennon,165.0,115,200,63.0,31,412,0,97,11,0
aaron-ramsey,178.0,68,150,76.0,39,260,0,42,31,1
abdelhamid-el-kaoutari,180.0,43,41,73.0,1,124,4,40,8,2


Although the original study shows there is little information in the EAT, IAT variables, we choose to use everything for now, and let the algorithm decide what's important or not.

If these values were to have any relevance, we'd expect them to be correlated with the number of cards a player gets: the higher the values, the more cards they'd get if they're darker in colour. So we add that correlation as a feature: i.e. the higher the correlation, the bigger the probability is they've a dark skin


In [10]:
c = group_players.corr()

In [11]:
for racism in ['meanIAT', 'meanExp']:
    for card in ['redCards', 'yellowCards', 'yellowReds']:
        a = c.loc[c.index.get_level_values(1)==racism, card].reset_index(level=1).fillna(value=0)
        players['cor_'+racism+card] = a[card]
players.head()

,height,defeats,victories,weight,goals,games,yellowReds,ties,yellowCards,redCards,cor_meanIATredCards,cor_meanIATyellowCards,cor_meanIATyellowReds,cor_meanExpredCards,cor_meanExpyellowCards,cor_meanExpyellowReds
playerShort,,,,,,,,,,,,,,,,
aaron-hughes,182.0,228,247,71.0,9,654,0,179,19,0,0.000000,-0.084032,0.000000,0.000000,-0.082210,0.000000
aaron-hunt,183.0,122,141,73.0,62,336,0,73,42,1,-0.055174,-0.162927,0.000000,-0.066079,-0.197755,0.000000
aaron-lennon,165.0,115,200,63.0,31,412,0,97,11,0,0.000000,-0.156448,0.000000,0.000000,-0.127625,0.000000
aaron-ramsey,178.0,68,150,76.0,39,260,0,42,31,1,-0.037021,-0.135808,0.000000,-0.071047,-0.161982,0.000000
abdelhamid-el-kaoutari,180.0,43,41,73.0,1,124,4,40,8,2,0.016348,0.035919,0.059003,0.000514,0.001129,0.040668


### Normalisation

First, we create extra features by normalizing to the number of games (always $\neq 0$).

Then we normalise all features

In [12]:
(players.games == 0).sum()

0

In [13]:
percentage_features = ['victories', 'ties', 'defeats', 'redCards', 'yellowReds', 'yellowCards']
for feature in percentage_features:
    players['percentage_'+feature] = players[feature] / players['games']
players.head()

,height,defeats,victories,weight,goals,games,yellowReds,ties,yellowCards,redCards,...,cor_meanIATyellowReds,cor_meanExpredCards,cor_meanExpyellowCards,cor_meanExpyellowReds,percentage_victories,percentage_ties,percentage_defeats,percentage_redCards,percentage_yellowReds,percentage_yellowCards
playerShort,,,,,,,,,,,,,,,,,,,,,
aaron-hughes,182.0,228,247,71.0,9,654,0,179,19,0,...,0.000000,0.000000,-0.082210,0.000000,0.377676,0.273700,0.348624,0.000000,0.000000,0.029052
aaron-hunt,183.0,122,141,73.0,62,336,0,73,42,1,...,0.000000,-0.066079,-0.197755,0.000000,0.419643,0.217262,0.363095,0.002976,0.000000,0.125000
aaron-lennon,165.0,115,200,63.0,31,412,0,97,11,0,...,0.000000,0.000000,-0.127625,0.000000,0.485437,0.235437,0.279126,0.000000,0.000000,0.026699
aaron-ramsey,178.0,68,150,76.0,39,260,0,42,31,1,...,0.000000,-0.071047,-0.161982,0.000000,0.576923,0.161538,0.261538,0.003846,0.000000,0.119231
abdelhamid-el-kaoutari,180.0,43,41,73.0,1,124,4,40,8,2,...,0.059003,0.000514,0.001129,0.040668,0.330645,0.322581,0.346774,0.016129,0.032258,0.064516


Whiten the data (0 mean, unit variance):

In [16]:
players = (players - players.mean()) / players.std()

### Categorisation

There are different views about categorical features (strings) since `sklearn` treats them as numbers. The best practice is to `one-hot encode` them but many argue that simply putting them to integer values doesn't affect the results as much. See [the discussion on the mailing list](https://www.mail-archive.com/search?l=scikit-learn-general%40lists.sourceforge.net&q=subject%3A%22Re%5C%3A+%5C%5BScikit%5C-learn%5C-general%5C%5D+Random+Forest+with+a+mix+of+categorical+and+lexical+features%22&o=newest&f=1)

We will follow the best practice and binarise them, since there are not that many values

---

Is is safe to keep the first values for `club`, `position` ?

In [28]:
# categories = group_players.agg(['club', 'leagueCountry', 'position']).first()
clubs     = group_players.apply(lambda x: len(x.club.unique()))     # get number of clubs
positions = group_players.apply(lambda x: len(x.position.unique())) # get number of positions
print("number of players with more than 1 club: "    , (clubs     != 1).sum())
print("number of players with more than 1 position: ", (positions != 1).sum())

number of players with more than 1 club:  0
number of players with more than 1 position:  0


Yes, it is

In [49]:
categories = group_players.agg({'club': 'first', 'leagueCountry':'first', 'position':'first'})
players_wcats = pd.get_dummies(players.join(categories))
players_wcats.columns.tolist()

# it's useful to see all the features now, but if long output is annoying for you as well, you can 
# click on the margin of the cell to toggle its scrolling (or double click to hide it ;) )

['height',
 'defeats',
 'victories',
 'weight',
 'goals',
 'games',
 'yellowReds',
 'ties',
 'yellowCards',
 'redCards',
 'cor_meanIATredCards',
 'cor_meanIATyellowCards',
 'cor_meanIATyellowReds',
 'cor_meanExpredCards',
 'cor_meanExpyellowCards',
 'cor_meanExpyellowReds',
 'percentage_victories',
 'percentage_ties',
 'percentage_defeats',
 'percentage_redCards',
 'percentage_yellowReds',
 'percentage_yellowCards',
 'club_1. FC Nürnberg',
 'club_1. FSV Mainz 05',
 'club_1899 Hoffenheim',
 'club_AC Ajaccio',
 'club_AS Nancy',
 'club_AS Saint-Étienne',
 'club_Arsenal FC',
 'club_Arsenal FC (R)',
 'club_Aston Villa',
 'club_Athletic Bilbao',
 'club_Atlético Madrid',
 'club_Bayer Leverkusen',
 'club_Bayern München',
 'club_Blackburn Rovers',
 'club_Bolton Wanderers',
 'club_Bor. Mönchengladbach',
 'club_Borussia Dortmund',
 'club_Bristol City',
 'club_CA Osasuna',
 'club_CF Badalona',
 'club_Celta Vigo',
 'club_Chelsea FC',
 'club_Crewe Alexandra',
 'club_Deportivo La Coruña',
 'club_ESTA

In [ ]:
le = preprocessing.LabelEncoder()

categorical_values = ['club', 'leagueCountry', 'position']
for name in categorical_values:
    categorie = group_players.agg({name:'first'})
    le.fit(categorie.as_matrix().flatten().tolist())
    players[name] = le.transform(categorie.as_matrix().flatten().tolist())

players.head()

In [ ]:
skin_color = group_players.agg({'rater1' : 'first'})
skin_color.head()

## Assignment 1: predict player's skin color

We convert the pandas data frame to lists in order to match the expected data format for scikit learn. We also map the player's skin color to an integer instead of a float.

In [ ]:
X = players.as_matrix()
Y = skin_color.as_matrix().flatten()
# map 0.25 to 1 etc
Y = np.array(list(map((lambda x: x*4), Y)))

Train the random forest using cross validation

In [ ]:
kf = KFold(n_splits=4)
clf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features=None)

for train_index, test_index in kf.split(X):
    clf = clf.fit(X[train_index], Y[train_index])
    # test model
    Y_predict = clf.predict(X[test_index])
    Y_predict2 = clf.predict(X[train_index])
    print("accurancy predictions test data: ",(Y[test_index] - Y_predict).tolist().count(0) / len(Y_predict))
    print("accurancy predictions training data: ",(Y[train_index] - Y_predict2).tolist().count(0) / len(Y_predict2))

In [ ]:
cross_val_score(clf, X, Y, scoring='accuracy', cv=4)

### Feature importance

In [ ]:
importances = clf.feature_importances_
std = np.std([clf.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1,  players.columns[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()